In [1]:
import DCCRN
import torch
from asteroid.data import DNSDataset
import os
import torch.nn.functional as F
import time
import torch.nn as nn
import shutil
import numpy as np
from framework import ABF, MultiResolutionSTFTLoss, SPKDLoss, build_review_kd, ReviewKD
from asteroid.models import DCCRNet
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import config as cfg
from tools_for_model import near_avg_index, max_index, min_index, Bar, cal_pesq, cal_stoi
from dataloader import create_dataloader
import inspect
import os
import feature_extraction
from asteroid.models import DCCRNet_mini
  
#os.environ["CUDA_VISIBLE_DEVICES"]="2"


In [2]:
model = DCCRN.DCCRN(rnn_units=cfg.rnn_units_student, masking_mode=cfg.masking_mode, use_clstm=cfg.use_clstm,kernel_num=cfg.kernel_num_student)
#model = DCCRN.DCCRN(rnn_units=cfg.rnn_units, masking_mode=cfg.masking_mode, use_clstm=cfg.use_clstm,kernel_num=cfg.kernel_num)
#data_set =  DNSDataset("/root/NTH_student/Speech_Enhancement_new/asteroid/egs/dns_challenge/baseline/data")
#model = DCCRNet.from_pretrained('JorisCos/DCCRNet_Libri1Mix_enhsingle_16k')
#model = DCCRNet_mini(architecture='DCCRN-CL-mini')
train_dataset =  DNSDataset("/root/NTH_student/train_loader")
train_loader = create_dataloader(mode='train',dataset=train_dataset)

In [3]:
feature_maps_encoder = []
feature_maps_decoder = []
feature_maps_lstm = []

In [4]:
def encoder_hook(module, input, output):
    feature_maps_encoder.append(output)
def decoder_hook(module, input, output):
    feature_maps_decoder.append(output)
def enhance_hook(module, input, output):
    feature_maps_lstm.append(output)



In [5]:
handler_encoder = [model.encoder[i].register_forward_hook(encoder_hook) for i in range(len(model.encoder))]
handler_decoder = [model.decoder[i].register_forward_hook(decoder_hook) for i in range(len(model.decoder))]
handler_lstm = model.enhance.register_forward_hook(enhance_hook)

In [6]:
X,Y = 0,0
i = 0
for x,y,_ in Bar(train_loader):
    if i == 1: break
    X=x
    Y=y
    i += 1

   16/60000: [>.................................................] - ETA 193.2s

In [7]:
h = model(X)

In [8]:
for i in range(len(feature_maps_encoder)):
    print(feature_maps_encoder[i].shape)
print("\n")
for i in range(len(feature_maps_decoder)):
    print(feature_maps_decoder[i].shape)
print("\n")
print(feature_maps_lstm[0][0].shape)
print(feature_maps_lstm[0][1].shape)

torch.Size([8, 8, 128, 483])
torch.Size([8, 16, 64, 483])
torch.Size([8, 32, 32, 483])
torch.Size([8, 64, 16, 483])
torch.Size([8, 64, 8, 483])
torch.Size([8, 64, 4, 483])


torch.Size([8, 64, 8, 484])
torch.Size([8, 64, 16, 484])
torch.Size([8, 32, 32, 484])
torch.Size([8, 16, 64, 484])
torch.Size([8, 8, 128, 484])
torch.Size([8, 2, 256, 484])


torch.Size([483, 8, 128])
torch.Size([483, 8, 128])


In [9]:
[handler_encoder[i].remove() for i in range(len(model.encoder)-1)]
[handler_decoder[i].remove() for i in range(len(model.decoder))]
handler_lstm.remove()

# DCCRN-CL

In [10]:
#model = DCCRNet.from_pretrained('JorisCos/DCCRNet_Libri1Mix_enhsingle_16k')
model = DCCRNet_mini(architecture='DCCRN-CL-test')

In [11]:
outputs_enc = []
outputs_dec= []
output_lstm = []

def hook_enc(module, input, output):
    outputs_enc.append(output.detach()) 
def hook_dec(module, input, output):
    outputs_dec.append(output.detach())
def hook_lstm(module, input, output):
    output_lstm.append(output.detach())

In [12]:
handler_encoder = [model.masker.encoders[i].register_forward_hook(hook_enc) for i in range(len(model.masker.encoders)-1)]
handler_decoder = [model.masker.decoders[i].register_forward_hook(hook_dec) for i in range(len(model.masker.decoders))]
handler_lstm = model.masker.encoders[-1].rnn.rnns[1].register_forward_hook(hook_lstm)

In [13]:
h = model(X)

In [14]:
for i in range(len(outputs_enc)):
    print(outputs_enc[i].shape)
print("\n")
for i in range(len(outputs_dec)):
    print(outputs_dec[i].shape)
print("\n")
print(output_lstm[0].shape)

torch.Size([8, 4, 128, 476])
torch.Size([8, 8, 64, 475])
torch.Size([8, 16, 32, 474])
torch.Size([8, 32, 16, 473])
torch.Size([8, 32, 8, 472])
torch.Size([8, 32, 4, 471])


torch.Size([8, 32, 4, 471])
torch.Size([8, 32, 8, 472])
torch.Size([8, 32, 16, 473])
torch.Size([8, 16, 32, 474])
torch.Size([8, 8, 64, 475])
torch.Size([8, 4, 128, 476])


torch.Size([8, 471, 32])
